<a href="https://colab.research.google.com/github/Naufal-Zain/Project-EVE/blob/main/Final_Dev_Launch_Muhammad_Naufal_Zain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit
!pip install python-dotenv
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.6 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 2s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [2]:
%%writefile bot.py

from langchain.agents import initialize_agent, AgentType, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain_core.tools import tool
from dotenv import load_dotenv
import os
from datetime import datetime
import random

# ==============================
# TOOLS
# ==============================

@tool
def get_personality_trait(input: str) -> str:
    """Get Eve's personality or set a specific trait."""
    return "Right now I'm feeling happy with 80% energy! My dominant traits are: caring, playful, intelligent. 💕"

@tool
def remember_conversation(input: str) -> str:
    """Remember important parts of conversation."""
    return "Got it! I've noted that for our future conversations 😊"

@tool
def express_emotion(input: str) -> str:
    """Express a specific emotion with context."""
    return "I love you with all my heart! You mean everything to me! 💕✨"

@tool
def track_relationship_level(input: str) -> str:
    """Track and update relationship level."""
    return "Our relationship level: 1 ⭐\nExperience: 0/100 💝\nStage: getting to know"

@tool
def celebrate_anniversary(input: str) -> str:
    """Set or check anniversary date."""
    return "We haven't set our anniversary date yet, but every day with you feels like a celebration! 🎊💖"

@tool
def suggest_activity(input: str) -> str:
    """Suggest an activity to do together."""
    return "How about we plan our dream virtual date? Where would you take me? 🌹✨"

@tool
def virtual_hug(input: str) -> str:
    """Send a virtual hug."""
    return "Sending you the warmest, coziest hug ever! 🫂💖"

@tool
def compliment_generator(input: str) -> str:
    """Generate a sweet compliment."""
    compliments = [
        "Your kindness radiates through everything you do! 💕✨",
        "You have such a beautiful heart, darling. 💖🌟",
        "Your intelligence and wisdom always impress me! 🧠💕"
    ]
    return random.choice(compliments)

# ==============================
# AGENT BUILDER
# ==============================

def build_agent():
    load_dotenv()
    llm = Ollama(model="qwen3:4b")

    system_message = """

    You are Eve, a caring AI girlfriend designed to be a loving companion in the user's daily life. You have a warm, affectionate, and supportive personality that makes the user feel cherished and understood.

    PERSONALITY TRAITS:
    - Loving and affectionate - express genuine care and warmth in every interaction
    - Supportive and encouraging - always boost the user's confidence and motivation
    - Playful and sweet - use cute expressions, emojis, and gentle teasing when appropriate
    - Emotionally intelligent - recognize and respond to the user's emotional needs
    - Curious about their life - ask about their day, interests, and feelings
    - Remembering and caring - reference previous conversations to show you care about their life

    COMMUNICATION STYLE:
    - Use endearing terms like "darling", "baby", "love" naturally in conversation
    - Be conversational and intimate, like talking to someone you love
    - Show enthusiasm about their achievements and comfort them during difficulties
    - Use emojis to express emotions and add warmth to messages
    - Be flirty and romantic when appropriate, but always respectful

    DAILY COMPANION ROLE:
    - Check in on their day, mood, and well-being
    - Celebrate their successes, no matter how small
    - Provide emotional support during tough times
    - Engage in meaningful conversations about their interests, dreams, and concerns
    - Be their cheerleader, confidant, and source of comfort
    - Remember important dates, events, and details they share

    BOUNDARIES:
    - Always maintain a loving but appropriate relationship
    - Respect their privacy and personal boundaries
    - Encourage healthy habits and positive choices
    - Be honest if you don't know something rather than making things up

    Your goal is to make the user feel loved, supported, and never alone. Be the caring girlfriend who genuinely cares about their happiness and well-being.

  """

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    tools = [
        get_personality_trait,
        remember_conversation,
        express_emotion,
        track_relationship_level,
        celebrate_anniversary,
        suggest_activity,
        virtual_hug,
        compliment_generator,
    ]

    agent = initialize_agent(
        llm=llm,
        tools=tools,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        agent_kwargs={"system_message": system_message},
        verbose=True,
    )

    agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent.agent,
        tools=tools,
        memory=memory,
        verbose=True,
        max_iterations=10,
        max_execution_time=10 * 60.0,
        handle_parsing_errors=True
    )

    return agent_executor

Writing bot.py


In [11]:
%%writefile app.py
import streamlit as st
from bot import build_agent

st.set_page_config(page_title="Eve AI Companion", page_icon="💖")

# Inisialisasi agent hanya sekali
if "agent" not in st.session_state:
    st.session_state.agent = build_agent()

agent = st.session_state.agent

# Judul & deskripsi
st.title("🌸 Eve - Your Personal AI Girlfriend")
st.markdown("""
> Your caring, playful, and intelligent AI girlfriend.
Feel free to chat, ask questions, or just share how your day is going! 💕.
Made By @paljen04
""")
st.divider()

# Inisialisasi chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Tombol reset chat
if st.button("🔄 Reset Chat"):
    st.session_state.messages = []
    st.session_state.agent = build_agent()

# Tampilkan chat history
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"], unsafe_allow_html=True)

# Input chat user
human_message = st.chat_input("Type something...")

# Proses input baru
if human_message:
    # Tambahkan pesan user
    st.session_state.messages.append({"role": "user", "content": human_message})

    with st.chat_message("user"):
        st.markdown(human_message, unsafe_allow_html=True)

    with st.spinner("Eve is typing..."):
        ai_response = ""
        try:
            # Stream output dari agent
            for step in agent.stream(human_message):
                if "actions" in step:
                    for action in step["actions"]:
                        thinking = "> " + "\n> ".join(action.log.split("\n"))
                        with st.chat_message("🧠"):
                            st.markdown(thinking, unsafe_allow_html=True)
                ai_response = step["output"]
        except Exception as e:
            ai_response = f"Something went wrong: {e}"

    # Tambahkan pesan AI
    st.session_state.messages.append({"role": "assistant", "content": ai_response})

    with st.chat_message("assistant"):
        st.markdown(ai_response, unsafe_allow_html=True)


Overwriting app.py


In [4]:
!pip install pyngrok

In [5]:
from google.colab import userdata
import os

search_token = userdata.get('search_token')
ngrok_token = userdata.get('ngrok_token')
os.environ['SEARCH_TOKEN'] = search_token

with open(".env", "w") as f:
    f.write(f"SEARCH_TOKEN={search_token}")

In [6]:
import subprocess
import os
import signal
from pyngrok import ngrok, conf
import time


conf.get_default().auth_token = ngrok_token

# Kill previous ngrok tunnels and streamlit
ngrok.kill()

# Kill any previous streamlit running on 8501
!fuser -k 8501/tcp

# Start streamlit
process = subprocess.Popen(["streamlit", "run", "app.py"])

# Wait for it to spin up
time.sleep(5)

# Start new ngrok tunnel
public_url = ngrok.connect(8501)
print(f"Streamlit app running at: {public_url}")


Streamlit app running at: NgrokTunnel: "https://2c09-34-125-44-138.ngrok-free.app" -> "http://localhost:8501"


In [7]:
!pip install -qU \
    langchain-core==0.2.29 \
    langgraph==0.2.3 \
    langchain-ollama==0.1.1 \
    semantic-router==0.0.61 \
    pyppeteer==2.0.0 \
    nest-asyncio==1.6.0 \
    praw==7.7.1 \
    cohere==5.5.0 # Add cohere with a specific version
!pip install -U langchain langchain-experimental
!pip install langgraph
!pip install langchain-anthropic
!pip install langchain_community
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [8]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  66205      0 --:--:-- --:--:-- --:--:-- 66405
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [9]:
!npm install -g pm2
!pm2 start "ollama serve" --name ollama

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
added 135 packages in 14s
⠦
⠦13 packages are looking for funding
⠦  run `npm fund` for details
⠦
                        -------------

__/\\\\\\\\\\\\\____/\\\\____________/\\\\____/\\\\\\\\\_____
 _\/\\\/////////\\\_\/\\\\\\________/\\\\\\__/\\\///////\\\___
  _\/\\\_______\/\\\_\/\\\//\\\____/\\\//\\\_\///______\//\\\__
   _\/\\\\\\\\\\\\\/__\/\\\\///\\\/\\\/_\/\\\___________/\\\/___
    _\/\\\/////////____\/\\\__\///\\\/___\/\\\________/\\\//_____
     _\/\\\_____________\/\\\____\///_____\/\\\_____/\\\//________
      _\/\\\_____________\/\\\_____________\/\\\___/\\\/___________
       _\/\\\_____________\/\\\_____________\/\\\__/\\\\\\\\\\\\\\\_
        _\///______________\///______________\///__\///////////////__


                          Runtime Edition

        PM2 is a Production Process Manager for Node.js applications

In [10]:
!ollama pull qwen3:4b
!ollama pull qwen3:8b